In [1]:
import os
import h5py
import itertools

import numpy as np
import pandas as pd

from tqdm import tqdm

In [2]:
def all_pairs(lst):
    if len(lst) < 2:
        yield []
        return
    if len(lst) % 2 == 1:
        # Handle odd length list
        for i in range(len(lst)):
            for result in all_pairs(lst[:i] + lst[i+1:]):
                yield result
    else:
        a = lst[0]
        for i in range(1, len(lst)):
            pair = (a,lst[i])
            for rest in all_pairs(lst[1:i]+lst[i+1:]):
                yield [pair] + rest

def Mjets(jets):
    # jets: 一個形狀為 (n, 4) 的 NumPy 陣列，其中 n 是噴射數量，每個噴射有四個屬性（pt, eta, phi, m）

    pt, eta, phi, m = jets.T  # 將噴射屬性分解為單獨的陣列

    px = pt * np.cos(phi)
    py = pt * np.sin(phi)
    pz = pt * np.sinh(eta)
    e = np.sqrt(m*m + px*px + py*py + pz*pz)

    return np.sqrt(e.sum()**2 - px.sum()**2 - py.sum()**2 - pz.sum()**2)

def PxPyPzE(jets):
    # jets: 一個形狀為 (n, 4) 的 NumPy 陣列，其中 n 是噴射數量，每個噴射有四個屬性（pt, eta, phi, m）
    pt, eta, phi, m = jets.T

    px = pt * np.cos(phi)
    py = pt * np.sin(phi)
    pz = pt * np.sinh(eta)
    e = np.sqrt(m*m + px*px + py*py + pz*pz)

    return px.sum(), py.sum(), pz.sum(), e.sum()

def PtEtaPhiM(px, py, pz, e):

    P = np.sqrt(px**2 + py**2 + pz**2)
    pt = np.sqrt(px**2 + py**2)
    eta = 1/2 * np.log((P + pz)/(P - pz))
    phi = np.arctan(py/px)
    m = np.sqrt(e**2 - px**2 - py**2 - pz**2)

    return pt, eta, phi, m

In [3]:
def get_Higgs_result(total_event, total_Higgs, correct_event, correct_Higgs, nh, nj):

    start_nj, end_nj = nj

    label = ['all' if nh == 'all' else f'{nh}h' for nj in range(start_nj, end_nj+2)]

    if nh == 'all':
        nh = slice(1, None)
        
    event_type = [f'Nj={nj}' for nj in range(start_nj, end_nj)]
    event_type.append(f'Nj>={end_nj}')
    event_type.append('Total')

    event_fraction = [total_event[nh, nj].sum() / total_event.sum() for nj in range(start_nj, end_nj)]
    event_fraction.append(total_event[nh, end_nj:].sum() / total_event.sum())
    event_fraction.append(total_event[nh].sum() / total_event.sum())

    event_efficiency = [correct_event[nh, nj].sum() / total_event[nh, nj].sum() for nj in range(start_nj, end_nj)]
    event_efficiency.append(correct_event[nh, end_nj:].sum() / total_event[nh, end_nj:].sum())
    event_efficiency.append(correct_event[nh].sum() / total_event[nh].sum())

    higgs_efficiency = [correct_Higgs[nh, nj].sum() / total_Higgs[nh, nj].sum() for nj in range(start_nj, end_nj)]
    higgs_efficiency.append(correct_Higgs[nh, end_nj:].sum() / total_Higgs[nh, end_nj:].sum())
    higgs_efficiency.append(correct_Higgs[nh].sum() / total_Higgs[nh].sum())
        
    result = {'Label':label,
              'Event type': event_type,
              'Event Fraction': event_fraction,
              'Event Efficiency': event_efficiency,
              'Higgs Efficiency': higgs_efficiency,
             }

    df = pd.DataFrame(result)

    return df

In [4]:
def chi2_triHiggs(m1, m2, m3):
    mh = 125.0
    return (m1 - mh)**2 + (m2 - mh)**2 + (m3 - mh)**2

def chi2_triHiggs_abs(m1, m2, m3):
    return abs(m1 - 119) + abs(m2 - 115) + abs(m3 - 111)

def compare_jet_list_triHiggs(pair1, pair2, nh_max=3):
    h1_true = {pair1[0], pair1[1]}
    h2_true = {pair1[2], pair1[3]}
    h3_true = {pair1[4], pair1[5]}  
    
    h1_test = {pair2[0], pair2[1]}
    h2_test = {pair2[2], pair2[3]}
    h3_test = {pair2[4], pair2[5]}
    
    test_h = [h1_test, h2_test, h3_test]
    
    nh = 0
    for id1, id2, id3 in itertools.permutations([0, 1, 2]):
        h1 = test_h[id1]
        h2 = test_h[id2]
        h3 = test_h[id3]
        if h1_true == h1:
            nh = 1
            if h2_true == h2:
                nh = 2
                if h3_true == h3:
                    nh = 3
                    break
                    
    same = True if nh==nh_max else False
    return same, nh

def compare_jet_list_triHiggs_optimized(pair1, pair2, nh_max=3):
    # 將pair1和pair2分別轉換為三個Higgs的集合
    h_true_sets = [{pair1[i], pair1[i+1]} for i in range(0, 6, 2)]
    h_test_sets = [{pair2[i], pair2[i+1]} for i in range(0, 6, 2)]
    
    # 計算匹配的Higgs數量
    nh = sum(1 for h_true in h_true_sets if h_true in h_test_sets)
    
    # 判斷是否所有Higgs都匹配
    same = nh == nh_max
    return same, nh

In [5]:
def get_particle_mask(quark_jet, particle_quarks):
    # quark_jet: 每個夸克對應的 jet 編號，shape 為 (n_event, 6)
    # particle_quarks: 粒子對應的夸克編號，shape 為 (n_quarks,)

    # 檢查是否每個夸克都有對應的 jet
    mask1 = np.all(quark_jet[:, particle_quarks] != -1, axis=1)

    # 對每一個事件，檢查每個夸克對應的 jet 都不重複
    count = np.array([[np.sum(event == event[i]) for i in particle_quarks] for event in quark_jet])
    mask2 = np.all(count == 1, axis=1)

    return mask1 & mask2

def chi2_method_triHiggs(file_path, use_btag=False):
    MAX_JETS = 15

    with h5py.File(file_path, 'r') as f:
        
        total_event = np.zeros((4, MAX_JETS + 1))
        total_Higgs = np.zeros((4, MAX_JETS + 1))
        correct_event = np.zeros((4, MAX_JETS + 1))
        correct_Higgs = np.zeros((4, MAX_JETS + 1))

        nevent = f['INPUTS/Source/pt'].shape[0]
        nevent = 1000

        for event in tqdm(range(nevent)):

            nj = f['INPUTS/Source/MASK'][event].sum()
            pt = f['INPUTS/Source/pt'][event]
            eta = f['INPUTS/Source/eta'][event]
            phi = f['INPUTS/Source/phi'][event]
            mass = f['INPUTS/Source/mass'][event]
            btag = f['INPUTS/Source/btag'][event]

            
            h1_b1 = f['TARGETS/h1/b1'][event] 
            h1_b2 = f['TARGETS/h1/b2'][event] 
            h2_b1 = f['TARGETS/h2/b1'][event] 
            h2_b2 = f['TARGETS/h2/b2'][event] 
            h3_b1 = f['TARGETS/h3/b1'][event] 
            h3_b2 = f['TARGETS/h3/b2'][event] 

            quark_jet = np.array([h1_b1, h1_b2, h2_b1, h2_b2, h3_b1, h3_b2]).reshape(1, 6)

            h1_mask = get_particle_mask(quark_jet, [0, 1])
            h2_mask = get_particle_mask(quark_jet, [2, 3])
            h3_mask = get_particle_mask(quark_jet, [4, 5])
            
            is_3h_event = h1_mask and h2_mask and h3_mask
            is_2h_event = (~h1_mask and h2_mask and h3_mask) or (h1_mask and ~h2_mask and h3_mask) or (h1_mask and h2_mask and ~h3_mask)
            is_1h_event = (~h1_mask and ~h2_mask and h3_mask) or (~h1_mask and h2_mask and ~h3_mask) or (h1_mask and ~h2_mask and ~h3_mask)
            is_0h_event = ~h1_mask and ~h2_mask and ~h3_mask
            
            if is_3h_event:
                total_event[3, nj] += 1
                total_Higgs[3, nj] += 3
            if is_2h_event:
                total_event[2, nj] += 1
                total_Higgs[2, nj] += 2   
            if is_1h_event:
                total_event[1, nj] += 1
                total_Higgs[1, nj] += 1   
            if is_0h_event:
                total_event[0, nj] += 1
                continue

            true_pair = [h1_b1,h1_b2, h2_b1,h2_b2, h3_b1,h3_b2]

            chisq = -1 
            pair = []

            jets_index = np.where(btag)[0] if use_btag else range(nj)

            for combination in itertools.combinations(jets_index, 6):
                for (i1,i2), (i3,i4), (i5,i6) in all_pairs(combination):       
                    jets = np.array([[pt[i], eta[i], phi[i], mass[i]] for i in [i1, i2, i3, i4, i5, i6]])

                    mh1, mh2, mh3 = Mjets(jets[[0, 1]]), Mjets(jets[[2, 3]]), Mjets(jets[[4, 5]])

                    tem = chi2_triHiggs(mh1, mh2, mh3)

                    if chisq < 0 or tem < chisq:
                        chisq = tem
                        pair = [i1, i2, i3, i4, i5, i6]

                        
            if is_3h_event:
                same, nh = compare_jet_list_triHiggs(true_pair, pair, nh_max=3)
                correct_Higgs[3, nj] += nh
                if same: correct_event[3, nj] += 1 
            if is_2h_event:
                same, nh = compare_jet_list_triHiggs(true_pair, pair, nh_max=2)
                correct_Higgs[2, nj] += nh
                if same: correct_event[2, nj] += 1     
            if is_1h_event:
                same, nh = compare_jet_list_triHiggs(true_pair, pair, nh_max=1)
                correct_Higgs[1, nj] += nh
                if same: correct_event[1, nj] += 1

        Nj = (6, 8)
        
        df_all = get_Higgs_result(total_event, total_Higgs, correct_event, correct_Higgs, nh='all', nj=Nj)
        print('All Events:')
        print(df_all)
        df_1h = get_Higgs_result(total_event, total_Higgs, correct_event, correct_Higgs, nh=1, nj=Nj)
        print('1 Higgs Events:')
        print(df_1h)
        df_2h = get_Higgs_result(total_event, total_Higgs, correct_event, correct_Higgs, nh=2, nj=Nj)
        print('2 Higgs Events:')
        print(df_2h)
        df_3h = get_Higgs_result(total_event, total_Higgs, correct_event, correct_Higgs, nh=3, nj=Nj)
        print('3 Higgs Events:')
        print(df_3h)
        
        file_name = os.path.splitext(os.path.basename(file_path))[0]
        
        frames = [df_all, df_1h, df_2h, df_3h]
        result = pd.concat(frames)
        
        result.to_csv(f'{file_name}--chisq_result.csv')

In [6]:
def get_Higgs_correct_fraction(events, nh, nj, jet_type='Nj'):
    # events: number of events in different categories (nh, nj, n_correct_h)
    start_nj, end_nj = nj

    label = [f'{nh}h' for _ in range(start_nj, end_nj+2)]

    total_event = events[nh].sum(axis=1)

    correct_3h_event = events[nh, :, 3]
    correct_2h_event = events[nh, :, 2]
    correct_1h_event = events[nh, :, 1]
    correct_0h_event = events[nh, :, 0]

    correct_Higgs = events[nh, :, 3] * 3 + events[nh, :, 2] * 2 + events[nh, :, 1]
        
    event_type = [f'{jet_type}={nj}' for nj in range(start_nj, end_nj)]
    event_type.append(f'{jet_type}>={end_nj}')
    event_type.append('Total')

    event_fraction = [total_event[nj] / total_event.sum() for nj in range(start_nj, end_nj)]
    event_fraction.append(total_event[end_nj:].sum() / total_event.sum())
    event_fraction.append(total_event.sum() / total_event.sum())


    eff_3h = [correct_3h_event[nj] / total_event[nj] for nj in range(start_nj, end_nj)]
    eff_3h.append(correct_3h_event[end_nj:].sum() / total_event[end_nj:].sum())
    eff_3h.append(correct_3h_event.sum() / total_event.sum())

    eff_2h = [correct_2h_event[nj] / total_event[nj] for nj in range(start_nj, end_nj)]
    eff_2h.append(correct_2h_event[end_nj:].sum() / total_event[end_nj:].sum())
    eff_2h.append(correct_2h_event.sum() / total_event.sum())

    eff_1h = [correct_1h_event[nj] / total_event[nj] for nj in range(start_nj, end_nj)]
    eff_1h.append(correct_1h_event[end_nj:].sum() / total_event[end_nj:].sum())
    eff_1h.append(correct_1h_event.sum() / total_event.sum())

    eff_0h = [correct_0h_event[nj] / total_event[nj] for nj in range(start_nj, end_nj)]
    eff_0h.append(correct_0h_event[end_nj:].sum() / total_event[end_nj:].sum())
    eff_0h.append(correct_0h_event.sum() / total_event.sum())

    eff_Higgs = [correct_Higgs[nj] / (total_event[nj] * nh) for nj in range(start_nj, end_nj)]
    eff_Higgs.append(correct_Higgs[end_nj:].sum() / (total_event[end_nj:].sum() * nh))
    eff_Higgs.append(correct_Higgs.sum() / (total_event.sum() * nh))
        
    result = {'Label':label,
              'Event type': event_type,
              'Event Fraction': event_fraction,
              '3h': eff_3h,
              '2h': eff_2h,
              '1h': eff_1h,
              '0h': eff_0h,
              'Higgs': eff_Higgs
             }

    df = pd.DataFrame(result)

    return df

def chi2_method_triHiggs_2(file_path, use_btag=False, pairing_method=chi2_triHiggs):
    MAX_JETS = 15

    with h5py.File(file_path, 'r') as f:
        
        # events: 總共有多少該類事件 (nh, nj, n_correct_h)
        events = np.zeros((4, MAX_JETS + 1, 4))
        events_nbj = np.zeros((4, MAX_JETS + 1, 4))

        nevent = min(10000, f['INPUTS/Source/pt'].shape[0])

        for event in tqdm(range(nevent)):

            nj = f['INPUTS/Source/MASK'][event].sum()
            pt = f['INPUTS/Source/pt'][event]
            eta = f['INPUTS/Source/eta'][event]
            phi = f['INPUTS/Source/phi'][event]
            mass = f['INPUTS/Source/mass'][event]
            btag = f['INPUTS/Source/btag'][event]
            nbj = btag.sum()

            
            h1_b1 = f['TARGETS/h1/b1'][event] 
            h1_b2 = f['TARGETS/h1/b2'][event] 
            h2_b1 = f['TARGETS/h2/b1'][event] 
            h2_b2 = f['TARGETS/h2/b2'][event] 
            h3_b1 = f['TARGETS/h3/b1'][event] 
            h3_b2 = f['TARGETS/h3/b2'][event] 

            quark_jet = np.array([h1_b1, h1_b2, h2_b1, h2_b2, h3_b1, h3_b2]).reshape(1, 6)

            h1_mask = get_particle_mask(quark_jet, [0, 1])
            h2_mask = get_particle_mask(quark_jet, [2, 3])
            h3_mask = get_particle_mask(quark_jet, [4, 5])

            event_h = [h1_mask, h2_mask, h3_mask].count(True)

            true_pair = [h1_b1,h1_b2, h2_b1,h2_b2, h3_b1,h3_b2]

            chisq = -1 
            pair = []

            if use_btag:
                if nbj >= 6:
                    jets_index = np.where(btag)[0][:6]
                elif nbj == 5:
                    jets_index = np.concatenate([np.where(btag)[0], np.where(~btag)[0][:1]])
                elif nbj == 4:
                    jets_index = np.concatenate([np.where(btag)[0], np.where(~btag)[0][:2]])
            else:
                jets_index = range(nj)

            for combination in itertools.combinations(jets_index, 6):
                for (i1,i2), (i3,i4), (i5,i6) in all_pairs(combination):       
                    jets = np.array([[pt[i], eta[i], phi[i], mass[i]] for i in [i1, i2, i3, i4, i5, i6]])

                    pt1, _, _, mh1 = PtEtaPhiM(*PxPyPzE(jets[[0, 1]]))
                    pt2, _, _, mh2 = PtEtaPhiM(*PxPyPzE(jets[[2, 3]]))
                    pt3, _, _, mh3 = PtEtaPhiM(*PxPyPzE(jets[[4, 5]]))

                    pt_mh_pairs = sorted(zip([pt1, pt2, pt3], [mh1, mh2, mh3]))
                    pt_sorted, mh_sorted = zip(*pt_mh_pairs)

                    mh1, mh2, mh3 = mh_sorted[::-1]

                    tem = pairing_method(mh1, mh2, mh3)

                    if chisq < 0 or tem < chisq:
                        chisq = tem
                        pair = [i1, i2, i3, i4, i5, i6]

            if event_h == 3:
                _, nh = compare_jet_list_triHiggs(true_pair, pair, nh_max=3)
                events[3, nj, nh] += 1
                events_nbj[3, nbj, nh] += 1
            elif event_h == 2:
                _, nh = compare_jet_list_triHiggs(true_pair, pair, nh_max=2)
                events[2, nj, nh] += 1
                events_nbj[2, nbj, nh] += 1
            elif event_h == 1:
                _, nh = compare_jet_list_triHiggs(true_pair, pair, nh_max=1)
                events[1, nj, nh] += 1
                events_nbj[1, nbj, nh] += 1
            elif event_h == 0:
                events[0, nj, 0] += 1
                events_nbj[0, nbj, 0] += 1
        
        
        df_3h = get_Higgs_correct_fraction(events, nh=3, nj=(6, 8), jet_type='Nj')  
        print('3 Higgs Events:')
        print(df_3h)

        df_style = df_3h.style.format({
            'Event Fraction': '{:.3f}',
            '3h': '{:.3f}',
            '2h': '{:.3f}',
            '1h': '{:.3f}',
            '0h': '{:.3f}',
            'Higgs': '{:.3f}'
        })
        print(df_style.to_latex(column_format='c|cccc|c'))


        df_3h_nbj = get_Higgs_correct_fraction(events_nbj, nh=3, nj=(4, 6), jet_type='Nbj')
        print('3 Higgs Events (nbj):')
        print(df_3h_nbj)

        df_style_nbj = df_3h_nbj.style.format({
            'Event Fraction': '{:.3f}',
            '3h': '{:.3f}',
            '2h': '{:.3f}',
            '1h': '{:.3f}',
            '0h': '{:.3f}',
            'Higgs': '{:.3f}'
        })
        print(df_style_nbj.to_latex(column_format='c|cccc|c'))



def SPANet_pairing_triHiggs(true_file, predict_file, nevent_max=50000):
    MAX_JETS = 15

    with h5py.File(true_file, 'r') as f_true, h5py.File(predict_file, 'r') as f_pre:
        
        # events: 總共有多少該類事件 (nh, nj, n_correct_h)
        events = np.zeros((4, MAX_JETS + 1, 4))

        nevent = min(f_true['INPUTS/Source/pt'].shape[0], nevent_max)

        for event in tqdm(range(nevent)):

            nj = f_true['INPUTS/Source/MASK'][event].sum()

            h1_b1 = f_true['TARGETS/h1/b1'][event] 
            h1_b2 = f_true['TARGETS/h1/b2'][event]
            h2_b1 = f_true['TARGETS/h2/b1'][event] 
            h2_b2 = f_true['TARGETS/h2/b2'][event]
            h3_b1 = f_true['TARGETS/h3/b1'][event]
            h3_b2 = f_true['TARGETS/h3/b2'][event]

            quark_jet = np.array([h1_b1, h1_b2, h2_b1, h2_b2, h3_b1, h3_b2]).reshape(1, 6)

            h1_mask = get_particle_mask(quark_jet, [0, 1])
            h2_mask = get_particle_mask(quark_jet, [2, 3])
            h3_mask = get_particle_mask(quark_jet, [4, 5])
            
            is_3h_event = h1_mask and h2_mask and h3_mask
            is_2h_event = (~h1_mask and h2_mask and h3_mask) or (h1_mask and ~h2_mask and h3_mask) or (h1_mask and h2_mask and ~h3_mask)
            is_1h_event = (~h1_mask and ~h2_mask and h3_mask) or (~h1_mask and h2_mask and ~h3_mask) or (h1_mask and ~h2_mask and ~h3_mask)
            is_0h_event = ~h1_mask and ~h2_mask and ~h3_mask

            true_pair = [h1_b1,h1_b2, h2_b1,h2_b2, h3_b1,h3_b2]

            pair = [f_pre['TARGETS/h1/b1'][event],
                    f_pre['TARGETS/h1/b2'][event],
                    f_pre['TARGETS/h2/b1'][event],
                    f_pre['TARGETS/h2/b2'][event],
                    f_pre['TARGETS/h3/b1'][event],
                    f_pre['TARGETS/h3/b2'][event]]

            if is_3h_event:
                _, nh = compare_jet_list_triHiggs(true_pair, pair, nh_max=3)
                events[3, nj, nh] += 1
            if is_2h_event:
                _, nh = compare_jet_list_triHiggs(true_pair, pair, nh_max=2)
                events[2, nj, nh] += 1   
            if is_1h_event:
                _, nh = compare_jet_list_triHiggs(true_pair, pair, nh_max=1)
                events[1, nj, nh] += 1
            if is_0h_event:
                events[0, nj, 0] += 1

        Nj = (6, 8)
        
        df_3h = get_Higgs_correct_fraction(events, nh=3, nj=Nj)
        print('3 Higgs Events:')
        print(df_3h)
        print(df_3h.to_latex(index=False, column_format='c|cccc|c', float_format='{:0.3f}'.format))


In [18]:
chi2_method_triHiggs_2('/home/r10222035/SPANet2/data/triHiggs/triHiggs_0b_3h_test.h5', use_first_6_bjet=False)

TypeError: chi2_method_triHiggs_2() got an unexpected keyword argument 'use_first_6_bjet'

In [10]:
chi2_method_triHiggs_2('Sample/SPANet/triHiggs_6b_3h_test.h5', use_first_6_bjet=True)

100%|██████████| 1000/1000 [00:04<00:00, 208.47it/s]

3 Higgs Events:
  Label Event type        3h        2h        1h        0h     Higgs
0    3h       Nj=6  0.500000  0.000000  0.123529  0.376471  0.541176
1    3h       Nj=7  0.322464  0.007246  0.184783  0.485507  0.388889
2    3h      Nj>=8  0.238411  0.026490  0.155629  0.579470  0.307947
3    3h      Total  0.328877  0.013369  0.159091  0.498663  0.390820


In [11]:
chi2_method_triHiggs_2('Sample/SPANet/triHiggs_6b_3h_test.h5', use_first_6_bjet=True, pairing_method=chi2_triHiggs_abs)

100%|██████████| 1000/1000 [00:04<00:00, 209.91it/s]

3 Higgs Events:
  Label Event type        3h        2h        1h        0h     Higgs
0    3h       Nj=6  0.476471  0.000000  0.135294  0.388235  0.521569
1    3h       Nj=7  0.293478  0.010870  0.184783  0.510870  0.362319
2    3h      Nj>=8  0.215232  0.026490  0.168874  0.589404  0.289183
3    3h      Total  0.303476  0.014706  0.167112  0.514706  0.368984


In [9]:
chi2_method_triHiggs_2('../SPANet2/data/triHiggs/gghhh_4b_PT40_3h_test.h5', use_btag=True)

  0%|          | 0/10000 [00:00<?, ?it/s]

100%|██████████| 10000/10000 [00:25<00:00, 384.63it/s]


3 Higgs Events:
  Label Event type  Event Fraction        3h        2h        1h        0h  \
0    3h       Nj=6        0.212901  0.516637  0.000000  0.098074  0.385289   
1    3h       Nj=7        0.326622  0.444064  0.006849  0.135845  0.413242   
2    3h      Nj>=8        0.460477  0.309312  0.007287  0.166802  0.516599   
3    3h      Total        1.000000  0.397465  0.005593  0.142058  0.454884   

      Higgs  
0  0.549329  
1  0.493912  
2  0.369771  
3  0.448546  
\begin{tabular}{c|cccc|c}
 & Label & Event type & Event Fraction & 3h & 2h & 1h & 0h & Higgs \\
0 & 3h & Nj=6 & 0.213 & 0.517 & 0.000 & 0.098 & 0.385 & 0.549 \\
1 & 3h & Nj=7 & 0.327 & 0.444 & 0.007 & 0.136 & 0.413 & 0.494 \\
2 & 3h & Nj>=8 & 0.460 & 0.309 & 0.007 & 0.167 & 0.517 & 0.370 \\
3 & 3h & Total & 1.000 & 0.397 & 0.006 & 0.142 & 0.455 & 0.449 \\
\end{tabular}



In [15]:
chi2_method_triHiggs_2('./Sample/SPANet/gghhh_4b_PT40_3h_new.h5', use_btag=True)

100%|██████████| 9944/9944 [00:48<00:00, 206.24it/s]

3 Higgs Events:
  Label Event type  Event Fraction        3h        2h        1h        0h  \
0    3h       Nj=6        0.242257  0.520133  0.000000  0.112080  0.367787   
1    3h       Nj=7        0.335278  0.277445  0.006599  0.182364  0.533593   
2    3h      Nj>=8        0.422466  0.160676  0.008331  0.189717  0.641276   
3    3h      Total        1.000000  0.286907  0.005732  0.168443  0.538918   

      Higgs  
0  0.557493  
1  0.342631  
2  0.229469  
3  0.346876  
\begin{tabular}{c|cccc|c}
 & Label & Event type & Event Fraction & 3h & 2h & 1h & 0h & Higgs \\
0 & 3h & Nj=6 & 0.242 & 0.520 & 0.000 & 0.112 & 0.368 & 0.557 \\
1 & 3h & Nj=7 & 0.335 & 0.277 & 0.007 & 0.182 & 0.534 & 0.343 \\
2 & 3h & Nj>=8 & 0.422 & 0.161 & 0.008 & 0.190 & 0.641 & 0.229 \\
3 & 3h & Total & 1.000 & 0.287 & 0.006 & 0.168 & 0.539 & 0.347 \\
\end{tabular}



In [7]:
chi2_method_triHiggs_2('./Sample/SPANet/pphhh_sm_4b_PT40.h5', use_btag=True)

  0%|          | 0/10000 [00:00<?, ?it/s]

100%|██████████| 10000/10000 [00:52<00:00, 191.86it/s]


3 Higgs Events:
  Label Event type  Event Fraction        3h        2h        1h        0h  \
0    3h       Nj=6        0.168849  0.555735  0.000000  0.106624  0.337641   
1    3h       Nj=7        0.310693  0.333626  0.007902  0.153644  0.504829   
2    3h      Nj>=8        0.520458  0.192872  0.006813  0.204403  0.595912   
3    3h      Total        1.000000  0.297872  0.006001  0.172122  0.524004   

      Higgs  
0  0.591276  
1  0.390108  
2  0.265549  
3  0.359247  
\begin{tabular}{c|cccc|c}
 & Label & Event type & Event Fraction & 3h & 2h & 1h & 0h & Higgs \\
0 & 3h & Nj=6 & 0.169 & 0.556 & 0.000 & 0.107 & 0.338 & 0.591 \\
1 & 3h & Nj=7 & 0.311 & 0.334 & 0.008 & 0.154 & 0.505 & 0.390 \\
2 & 3h & Nj>=8 & 0.520 & 0.193 & 0.007 & 0.204 & 0.596 & 0.266 \\
3 & 3h & Total & 1.000 & 0.298 & 0.006 & 0.172 & 0.524 & 0.359 \\
\end{tabular}



In [10]:
chi2_method_triHiggs_2('../SPANet2/data/triHiggs/gghhh_4b_PT40_3h_test.h5', use_btag=True, pairing_method=chi2_triHiggs_abs)

100%|██████████| 10000/10000 [00:52<00:00, 191.03it/s]

3 Higgs Events:
  Label Event type  Event Fraction        3h        2h        1h        0h  \
0    3h       Nj=6          0.2413  0.456693  0.000000  0.130957  0.412350   
1    3h       Nj=7          0.3347  0.239020  0.009262  0.183747  0.567971   
2    3h      Nj>=8          0.4240  0.135849  0.006604  0.196698  0.660849   
3    3h      Total          1.0000  0.247800  0.005900  0.176500  0.569800   

      Higgs  
0  0.500345  
1  0.306444  
2  0.205818  
3  0.310567  
\begin{tabular}{c|cccc|c}
 & Label & Event type & Event Fraction & 3h & 2h & 1h & 0h & Higgs \\
0 & 3h & Nj=6 & 0.241 & 0.457 & 0.000 & 0.131 & 0.412 & 0.500 \\
1 & 3h & Nj=7 & 0.335 & 0.239 & 0.009 & 0.184 & 0.568 & 0.306 \\
2 & 3h & Nj>=8 & 0.424 & 0.136 & 0.007 & 0.197 & 0.661 & 0.206 \\
3 & 3h & Total & 1.000 & 0.248 & 0.006 & 0.176 & 0.570 & 0.311 \\
\end{tabular}



In [7]:
chi2_method_triHiggs_2('./Sample/SPANet/gghhh_4b_PT40_3h_new.h5', use_btag=True, pairing_method=chi2_triHiggs_abs)

  0%|          | 0/9944 [00:00<?, ?it/s]

100%|██████████| 9944/9944 [00:47<00:00, 210.78it/s]


3 Higgs Events:
  Label Event type  Event Fraction        3h        2h        1h        0h  \
0    3h       Nj=6        0.242257  0.449979  0.000000  0.131590  0.418431   
1    3h       Nj=7        0.335278  0.249250  0.007499  0.177265  0.565987   
2    3h      Nj>=8        0.422466  0.144965  0.008569  0.193525  0.652940   
3    3h      Total        1.000000  0.253821  0.006134  0.173069  0.566975   

      Higgs  
0  0.493843  
1  0.313337  
2  0.215187  
3  0.315601  
\begin{tabular}{c|cccc|c}
 & Label & Event type & Event Fraction & 3h & 2h & 1h & 0h & Higgs \\
0 & 3h & Nj=6 & 0.242 & 0.450 & 0.000 & 0.132 & 0.418 & 0.494 \\
1 & 3h & Nj=7 & 0.335 & 0.249 & 0.007 & 0.177 & 0.566 & 0.313 \\
2 & 3h & Nj>=8 & 0.422 & 0.145 & 0.009 & 0.194 & 0.653 & 0.215 \\
3 & 3h & Total & 1.000 & 0.254 & 0.006 & 0.173 & 0.567 & 0.316 \\
\end{tabular}

3 Higgs Events (nbj):
  Label Event type  Event Fraction        3h        2h        1h        0h  \
0    3h      Nbj=4        0.309332  0.189857  0.00

In [8]:
chi2_method_triHiggs_2('./Sample/SPANet/pphhh_sm_4b_PT40.h5', use_btag=True, pairing_method=chi2_triHiggs_abs)

100%|██████████| 10000/10000 [00:48<00:00, 206.44it/s]

3 Higgs Events:
  Label Event type  Event Fraction        3h        2h        1h        0h  \
0    3h       Nj=6        0.168849  0.651050  0.000000  0.095315  0.253635   
1    3h       Nj=7        0.310693  0.401229  0.014047  0.158033  0.426690   
2    3h      Nj>=8        0.520458  0.225891  0.009434  0.212788  0.551887   
3    3h      Total        1.000000  0.352155  0.009274  0.175941  0.462630   

      Higgs  
0  0.682822  
1  0.463272  
2  0.303110  
3  0.416985  
\begin{tabular}{c|cccc|c}
 & Label & Event type & Event Fraction & 3h & 2h & 1h & 0h & Higgs \\
0 & 3h & Nj=6 & 0.169 & 0.651 & 0.000 & 0.095 & 0.254 & 0.683 \\
1 & 3h & Nj=7 & 0.311 & 0.401 & 0.014 & 0.158 & 0.427 & 0.463 \\
2 & 3h & Nj>=8 & 0.520 & 0.226 & 0.009 & 0.213 & 0.552 & 0.303 \\
3 & 3h & Total & 1.000 & 0.352 & 0.009 & 0.176 & 0.463 & 0.417 \\
\end{tabular}

3 Higgs Events (nbj):
  Label Event type  Event Fraction        3h        2h        1h        0h  \
0    3h      Nbj=4        0.300600  0.294918  0.01

## SPANet pairing

In [9]:
true_file = '/home/r10222035/SPANet2/data/triHiggs/gghhh_0b_test.h5'
predict_file = '/home/r10222035/SPANet2/data/triHiggs/gghhh_0b_test-predict.h5'
SPANet_pairing_triHiggs(true_file, predict_file)

100%|██████████| 52290/52290 [02:27<00:00, 353.85it/s]

1 Higgs Events:
  Label Event type   3h   2h        1h        0h     Higgs
0    1h       Nj=6  0.0  0.0  0.162483  0.837517  0.162483
1    1h       Nj=7  0.0  0.0  0.150334  0.849666  0.150334
2    1h      Nj>=8  0.0  0.0  0.145794  0.854206  0.145794
3    1h      Total  0.0  0.0  0.155971  0.844029  0.155971
2 Higgs Events:
  Label Event type   3h        2h        1h        0h     Higgs
0    2h       Nj=6  0.0  0.054968  0.288859  0.656173  0.199397
1    2h       Nj=7  0.0  0.051811  0.258854  0.689334  0.181239
2    2h      Nj>=8  0.0  0.034583  0.247899  0.717518  0.158533
3    2h      Total  0.0  0.050102  0.271850  0.678048  0.186027
3 Higgs Events:
  Label Event type        3h        2h        1h        0h     Higgs
0    3h       Nj=6  0.532476  0.000000  0.119234  0.348290  0.572220
1    3h       Nj=7  0.344528  0.020845  0.165605  0.469021  0.413627
2    3h      Nj>=8  0.237457  0.021993  0.186254  0.554296  0.314204
3    3h      Total  0.374620  0.014239  0.156423  0.454717  0

In [7]:
true_file = '/home/r10222035/SPANet2/data/triHiggs/triHiggs_0b_test.h5'
predict_file = '/home/r10222035/SPANet2/data/triHiggs/triHiggs_0b_test-predict.h5'
SPANet_pairing_triHiggs(true_file, predict_file)

100%|██████████| 10000/10000 [00:23<00:00, 422.96it/s]


1 Higgs Events:
  Label Event type   3h   2h        1h        0h     Higgs
0    1h       Nj=6  0.0  0.0  0.168159  0.831841  0.168159
1    1h       Nj=7  0.0  0.0  0.171756  0.828244  0.171756
2    1h      Nj>=8  0.0  0.0  0.122257  0.877743  0.122257
3    1h      Total  0.0  0.0  0.161255  0.838745  0.161255
2 Higgs Events:
  Label Event type   3h        2h        1h        0h     Higgs
0    2h       Nj=6  0.0  0.082079  0.280438  0.637483  0.222298
1    2h       Nj=7  0.0  0.061630  0.276342  0.662028  0.199801
2    2h      Nj>=8  0.0  0.035211  0.253521  0.711268  0.161972
3    2h      Total  0.0  0.066535  0.274045  0.659420  0.203557
3 Higgs Events:
  Label Event type        3h        2h        1h        0h     Higgs
0    3h       Nj=6  0.656463  0.000000  0.081633  0.261905  0.683673
1    3h       Nj=7  0.436416  0.017341  0.167630  0.378613  0.503854
2    3h      Nj>=8  0.341727  0.017986  0.172662  0.467626  0.411271
3    3h      Total  0.478214  0.011983  0.141612  0.368192  0

In [12]:
true_file = '/home/r10222035/SPANet2/data/triHiggs/triHiggs_0b_3h_test.h5'
predict_file = '/home/r10222035/SPANet2/data/triHiggs/triHiggs_0b_3h_test-predict.h5'
SPANet_pairing_triHiggs(true_file, predict_file)

100%|██████████| 50000/50000 [01:56<00:00, 430.49it/s]

3 Higgs Events:
  Label Event type        3h        2h        1h        0h     Higgs
0    3h       Nj=6  0.679690  0.000000  0.084163  0.236147  0.707745
1    3h       Nj=7  0.476826  0.014456  0.149746  0.358972  0.536379
2    3h      Nj>=8  0.311074  0.027201  0.184350  0.477375  0.390658
3    3h      Total  0.491466  0.013703  0.139114  0.355718  0.546972


In [13]:
true_file = '/home/r10222035/SPANet2/data/triHiggs/triHiggs_6b_3h_test.h5'
predict_file = '/home/r10222035/SPANet2/data/triHiggs/triHiggs_6b_3h_test-predict.h5'
SPANet_pairing_triHiggs(true_file, predict_file)

100%|██████████| 11521/11521 [00:26<00:00, 434.74it/s]

3 Higgs Events:
  Label Event type        3h        2h        1h        0h     Higgs
0    3h       Nj=6  0.679505  0.000000  0.074180  0.246315  0.704232
1    3h       Nj=7  0.522939  0.013298  0.126995  0.336769  0.574136
2    3h      Nj>=8  0.354066  0.019401  0.179743  0.446790  0.426914
3    3h      Total  0.492456  0.012535  0.135562  0.359448  0.546000


In [9]:
true_file = '/home/r10222035/SPANet2/data/triHiggs/gghhh_4b_PT40_test.h5'
predict_file = '/home/r10222035/SPANet2/data/triHiggs/gghhh_4b_PT40_test-predict.h5'
SPANet_pairing_triHiggs(true_file, predict_file)

100%|██████████| 50000/50000 [01:54<00:00, 435.21it/s]

3 Higgs Events:
  Label Event type        3h        2h        1h        0h     Higgs
0    3h       Nj=6  0.619405  0.000000  0.097283  0.283312  0.651833
1    3h       Nj=7  0.480795  0.007947  0.150047  0.361211  0.536108
2    3h      Nj>=8  0.349703  0.010682  0.171068  0.468546  0.413848
3    3h      Total  0.458905  0.007174  0.146130  0.387791  0.512398
\begin{tabular}{c|cccc|c}
\toprule
Label & Event type &    3h &    2h &    1h &    0h &  Higgs \\
\midrule
   3h &       Nj=6 & 0.619 & 0.000 & 0.097 & 0.283 &  0.652 \\
   3h &       Nj=7 & 0.481 & 0.008 & 0.150 & 0.361 &  0.536 \\
   3h &      Nj>=8 & 0.350 & 0.011 & 0.171 & 0.469 &  0.414 \\
   3h &      Total & 0.459 & 0.007 & 0.146 & 0.388 &  0.512 \\
\bottomrule
\end{tabular}




/tmp/ipykernel_2615392/495308457.py:206: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(df_3h.to_latex(index=False, column_format='c|cccc|c', float_format='{:0.3f}'.format))
